In [3]:
import os
import sys

sys.path.append(os.path.abspath(".."))

import randomdataset


In [33]:
%%writefile randomschema.yaml

- name: participants
  typename: randomdataset.Dataset
  fields:
  - name: id
    typename: randomdataset.UIDFieldGen
  - name: FirstName
    typename: randomdataset.StrFieldGen
    lmin: 6
    lmax: 14
  - name: LastName
    typename: randomdataset.StrFieldGen
    lmin: 6
    lmax: 14
  - name: Age
    typename: randomdataset.IntFieldGen
    vmin: 18
    vmax: 90
  - name: is_employed
    typename: randomdataset.BoolFieldGen
    as_string: False
        
- name: tests
  typename: randomdataset.Dataset
  fields:
  - name: id
    typename: randomdataset.UIDFieldGen
  - name: patient_id
    typename: randomdataset.UIDFieldGen
  - name: test_type
    typename: randomdataset.SetFieldGen
    field_type: str
    values: ["Type1", "Type2", "Unknown"]
  - name: location
    typename: randomdataset.StrFieldGen
  - name: result
    typename: randomdataset.IntFieldGen
    vmin: 0
    vmax: 3
  - name: value
    typename: randomdataset.FloatFieldGen
    vmin: 0
    vmax: 9

Overwriting randomschema.yaml


In [35]:
!generate_dataset randomschema.yaml .

Schema: '<unopened file 'randomschema.yaml' r>'
Output: 'C:\Users\Eric\workspace\RandomDataset\examples'
Generating 10 lines with 'randomdataset.generators.CSVGenerator'


In [23]:
%%writefile exeteraschema.json

{
  "exetera": {
    "version": "1.0.0"
  },
  "schema": {
    "participants": {
      "primary_keys": [
        "id"
      ],
      "fields": {
        "id": {
          "field_type": "fixed_string",
          "length": 32
        },
        "FirstName": {
          "field_type": "string"
        },
        "LastName": {
          "field_type": "string"
        },
        "Age": {
          "field_type": "numeric",
          "value_type": "int32"
        },
        "is_employed": {
          "field_type": "categorical",
          "categorical": {
            "value_type": "int8",
            "strings_to_values": {
              "False": 1,
              "True": 2
            }
          }
        }
      }
    },
    "tests": {
      "primary_keys": [
        "id"
      ],
      "foreign_keys": {
        "patient_id": {
          "space": "patients",
          "key": "id"
        }
      },
      "fields": {
        "id": {
          "field_type": "fixed_string",
          "length": 32
        },
        "patient_id": {
          "field_type": "fixed_string",
          "length": 32
        },
        "test_type": {
          "field_type": "categorical",
          "categorical": {
            "value_type": "int8",
            "strings_to_values": {
              "Unknown": 0,
              "Type1": 1,
              "Type2": 2
            }
          }
        },
        "location": {
          "field_type": "string"
        },
        "result": {
          "field_type": "numeric",
          "value_type": "int32"
        },
        "value": {
          "field_type": "numeric",
          "value_type": "float32"
        } 
      }
    } 
  }
}


Overwriting exeteraschema.json


In [24]:
import json
with open("exeteraschema.json") as o:
    json.load(o)

In [12]:
!exetera -s exeteraschema.json -i "participants:participants.csv, tests:tests.csv" -o dataset.hdf5

'exetera.exe' is not recognized as an internal or external command,
operable program or batch file.


In [28]:
import numpy as np
import datetime

dt=datetime.datetime(year=9999, month=1, day=1)
print(dt.isoformat())
# print((dt - datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)).total_seconds())
# valid_tos = np.full(10, dt.timestamp())

9999-01-01T00:00:00
